In [5]:
import boto3

#Creating Session With Boto3.
session = boto3.Session(
aws_access_key_id="",
aws_secret_access_key=""
)

#Creating S3 Resource From the Session.
s3 = session.resource('s3')

In [6]:
files = ['us-cities-demographics.csv','airport-codes_csv.csv','I94_SAS_Labels_Descriptions.SAS']

In [ ]:
for file in files:
    object = s3.Object('immigration-raw-data', file)
    result = object.put(Body=open(file, 'rb'))
    

In [9]:
object = s3.Object('immigration-raw-data', 'GlobalLandTemperaturesByCity.csv')
result = object.put(Body=open('../../data2/GlobalLandTemperaturesByCity.csv', 'rb'))

In [13]:
# import OS module
import os
 
# Get the list of all files and directories
path = "../../data/18-83510-I94-Data-2016/"
sas_files = os.listdir(path) 

../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat


In [16]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()
for file in sas_files:
    df_spark = spark.read.format('com.github.saurfang.sas.spark').load(path + file)
    df_spark.write.partitionBy("i94mon").mode('append').parquet("sas_data3")

In [17]:
df_spark=spark.read.parquet("sas_data3")

In [18]:
df_spark.count()

40790529

In [ ]:
!aws configure

AWS Access Key ID [None]: 

In [ ]:
!aws s3 cp sas_data3 s3://immigration-raw-data/immigration --recursive